# Zhang's Algorithm For Camera Calibration

### Import Statements

In [3]:
import os
from camera_callibration_helper import *
import cv2
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


### Load the Images
* raw_img_list (list): list of 40 BGR input images
* grey_img_list (list): list of 40 grey scale input images
* img_labels (list): list of 40 image filenames (mainly for debugging)

In [4]:
given_data_path = 'C:\\Users\jo_wang\Desktop\ECE661\HW08\Dataset1'
raw_img_list, grey_img_list, img_labels = loadImages(given_data_path)
assert(len(grey_img_list) == 40)
assert(len(raw_img_list) == 40)
assert(len(img_labels) == 40)

### Apply Canny Edge Detector On Grey Scale Images
* edge_img_list (list): list of edge maps from Canny

In [5]:
edge_img_list = performCanny(grey_img_list)
assert(len(edge_img_list) == 40)

### Apply Hough Transform To all the Images
* hough_lines_list (list): list of 40 images after applying hough transform

In [6]:
hough_lines_list = performHoughTransform(edge_img_list)
assert(len(hough_lines_list) == len(edge_img_list))

### Get the corner points from selected images
* all_corners (list): at each index, list of 80 corner points
* the_chosen_one (list): index of images to use

In [7]:
the_chosen_one = [0, 1, 3, 5, 6, 10, 21, 26, 27, 28, 29, 31, 33, 34, 35, 36, 39]
all_corners = list()
for i in the_chosen_one:
    h_lines, v_lines = get_Horizontal_Vert_Lines(hough_lines_list[i])

    v_lines = np.array(v_lines).reshape(-1,2)
    h_lines = np.array(h_lines).reshape(-1,2)

    img = deepcopy(raw_img_list[i])
    corner_points = getCorners(v_lines, h_lines)
    if len(corner_points) == 80:
        all_corners.append(corner_points)

    for j, point in enumerate(corner_points):
        try:
            img = cv2.circle(img, point, 3, (0, 0, 255), -1)
            cv2.putText(img, str(j), (point[0]+5, point[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        except OverflowError:
            pass

    # cv2.imwrite(f'points_{i}.jpg', img)

### Get world point coordinates
* world_points (list): list of 80 world point coordinates in sorted order

In [8]:
world_points = list()
for i in range(0, 200, 20):
    for j in range(0, 160, 20):
        world_points.append([i,j])


### Estimate Homographies between world points and all corners
* all_homographies (list): list of 3x3 homographies relating world points to each image
* DON'T DELETE THIS ONE CUZ THIS ONE IS URS

In [9]:
# def get_Ab(r2_points, projected_points):
#     A = list()
#     for i,j in zip(r2_points, projected_points):
#         r1 = i + [1] + [0, 0, 0] + [-i[0] * j[0], -i[1] * j[0]]
#         r2 = [0, 0, 0] + i + [1] + [-i[0] * j[1], -i[1] * j[1]]
#         A.append([r1, r2])
#
#     b = np.array(projected_points).reshape(-1, 1)
#
#     return np.array(A).reshape(-1, 8), b
#
#
# def get_H(world_points, corners):
#     A,b = get_Ab(world_points, corners)
#     print(A.shape)
#     H = list(np.linalg.solve(A.T @ A, A.T @ b).reshape(-1))
#     H.append(1)
#     return np.array(H).reshape(3, 3)

In [14]:

# generate rows with one point
def A_rows(X: list, X_prime: list) -> list:
    x = X + [1]
    r1 = x + [0, 0, 0] + [-X[0] * X_prime[0], -X[1] * X_prime[0]]
    r2 = [0, 0, 0] + x + [-X[0] * X_prime[1], -X[1] * X_prime[1]]

    return [r1, r2]

# generate matrix with n points
def get_A(r2_points, projected_points):
    if len(r2_points) != len(projected_points):
        sys.exit('Provide same number of points in both spaces')

    A = list(map(lambda x: A_rows(x[0], x[1]), zip(r2_points, projected_points)))

    return np.array(A).reshape(-1, 8)

# generate column with n points
def get_B(X_prime):
    return np.array(X_prime).reshape(-1, 1)

# gets H from given A and b using least squares
def get_H(A, b):
    H = list(np.linalg.solve(A.T @ A, A.T @ b).reshape(-1))
    H.append(1)
    return np.array(H).reshape(3, 3)

In [15]:
all_homographies = list()
for corners in all_corners:
    A = get_A(world_points, corners)
    b = get_B(corners)
    h = get_H(A, b)
    all_homographies.append(h)

print(all_homographies[0])

[[1.19015679e+00 2.58919279e-01 2.18092899e+02]
 [5.62934731e-01 2.20145028e-01 1.86137411e+02]
 [1.90814301e-04 4.11177197e-04 1.00000000e+00]]


### Compute V

In [ ]:
def get_V(i, j, h):
    v = np.zeros((6,1))
    i -= 1
    j -= 1

    v[0][0] = h[0][i] * h[0][j]
    v[1][0] = h[0][i] * h[1][j] + h[1][i] + h[0][j]
    v[2][0] = h[1][i] + h[1][j]
    v[3][0] = (h[2][i] * h[0][j]) + (h[0][i] * h[2][j])
    v[4][0] = (h[2][i] * h[1][j]) + (h[1][i] * h[2][j])
    v[5][0] = h[2][i] * h[2][j]

    return v


In [ ]:
Big_V = np.zeros((1,6))
for h in all_homographies:
    r1 = get_V(i=1, j=2, h=h).T
    r2 = get_V(i=1,j=1,h=h).T - get_V(i=2,j=2,h=h).T
    Big_V = np.vstack((Big_V, r1))
    Big_V = np.vstack((Big_V, r2))

Big_V = Big_V[1:, :]

u, s, vh = np.linalg.svd(Big_V)
b = vh[-1]

w = np.zeros((3,3))
w[0][0] = b[0]
w[0][1] = b[1]
w[0][2] = b[3]
w[1][0] = b[1]
w[1][1] = b[2]
w[1][2] = b[4]
w[2][0] = b[3]
w[2][1] = b[4]
w[2][2] = b[5]

